# FFT with our own C++ code and SWIG


We can take this opportunity to show you how to use your own C++ implementation of codes from within python. 

One caveat: we are using `vector<complex<double>>` for our C++ FFT implementation. However, the SWIG numpy setup does not natively support this type, so it's a bit beyond what I would like to show you. 

We will therefore just have to copy our back and forth (once) to use our software. 



## Step 1 : Look at C++ files

First we have our FFT C++ files: 


In [ ]:
! cat our_fft.cpp our_fft.hpp



## Step 2 : Look at SWIG interface file

The magic of SWIG is to create wrapper C++ functions that use the "cython" interface. We also have an "interface" file for SWIG :

In [ ]:
! cat our_fft.i

## Step 3 : Look at SWIG setup file

This tells SWIG to generate a file called ```our_fft_wrap.cxx``` using source from ```our_fft.cpp``` with C++11. 


In [ ]:
! cat setup_our_fft.py

Now create the interface using the interface file. This will create two files, ```example_wrap.cxx``` and ```example.py```. They are not intended to be human-readable, so we don't need to look into them. 

## Step 4: Create SWIG interface from file

In [ ]:
! swig -c++ -python our_fft.i 

## Step 5 : compile C++ and interface

This will compile our file ```our_fft.cpp``` along with ```our_fft_wrap.cxx``` using the rules we set (for instance, using the C++11 compiler). 

We are now free to use this in python!

In [ ]:
! python setup_our_fft.py build_ext --inplace


## Step 6: Make profit: 

First we import the path correctly. 

In [ ]:
import sys
import os

sys.path.append( os.path.abspath(".") )

print (sys.path)




Next we import our actual function from the automatically generated "our_fft.py"

In [ ]:
from our_fft import *


And finally, we can use our C++ code from within python!: 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

N = 1024 # number of samples
k = 10.0 # frequency / N
x = np.arange(N) 
# First we use numpy to make a sine wave
y = np.sin( -2. * np.pi * k /N * x ) 

# Now we have to copy the numpy array to a vector<complex<double>>. 
# This is suboptimal, but this type is not automatically supported
# by the swig numpy interface. We're just using this
# for demonstration anyway, though. 
y_ours = vector_complex_double( N )
for i in range(N):
    y_ours[i] = y[i]

Y_ours = our_fft(y_ours)  # Our C++ FFT implementation ... NOW WITH SWIG!

# And convert back to numpy
Y = np.zeros(N, dtype=complex)
for i in range(N):
    Y[i] = Y_ours[i]

Y_abs = abs(Y)
Ynp_abs = abs(Y)


# Success!

In [ ]:

f1 = plt.figure(1)
plt.plot( x, y )
plt.xlabel('x')
plt.ylabel('y')

f2 = plt.figure(2)
plt.plot( x, Y_abs, label="our SWIG fft" )
plt.plot( x, Ynp_abs, label="numpy fft")
plt.ylabel('Fourier Component')
plt.xlabel('Spectral Index')
plt.legend()
plt.xlim([0,20])

plt.show()